In [ ]:
using VCFTool

In [ ]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_example_file_path = joinpath(input_dir, "example.vcf.gz")

In [ ]:
vcf_738 = read_vcf(vcf_738_file_path)

vcf_example = read_vcf(vcf_example_file_path)

vcf_to_use = vcf_example

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

## Julia DataFrame Query 

In [ ]:
using Dates

start_time = now()

regions = Dict([
        ("chr2", 240630710),
        ("chr7", 150999023),
        ("chr10", 99836239),
        ("chr10", 99851537),
        ("chr16", 88646828),
        ("chr19", 51354484),
        ("chr21", 36146408),
        ])

query_regions(
    regions, 
    vcf_to_use,
    output_dir,
)

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

## JuliaDB

An unzipped VCF file can be loaded into an indextable or ndsparse array directly using loadtable() or ndsparese() respectively. However, its much faster to query variants if they are in an ndsparse array because the chromosome and position are both part of the index.

In [3]:
using CodecZlib: GzipDecompressorStream
using CSV
using JuliaDB

function make_ndsparse(vcf_gz_file_path::String)
    
    vcf_file_path = string(Array(split(vcf_gz_file_path, ".gz"))[1])

    if isfile(vcf_file_path) == false
        
        io = open(vcf_gz_file_path)

        io2 = GzipDecompressorStream(io)

        file = CSV.File(io2, comment="##", delim='\t', header=1)

        vcf_file_path = Array(split(vcf_gz_file_path, ".gz"))[1]

        CSV.write(vcf_file_path, delim='\t', file)

        close(io)

    end
    
    vcf_ndsparse = loadndsparse(
        vcf_file_path,
        delim='\t',
        header_exists=false,  
        colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
        indexcols=[:CHROM, :POS]
        )
    
    return vcf_ndsparse

end

make_ndsparse (generic function with 1 method)

In [4]:
vcf_ndsparse_738 = make_ndsparse("../input/738_variants.vcf.gz")

2-d NDSparse with 5217069 values (8 field named tuples):
    Dimensions#  colname  type
──────────────────
1  CHROM    String
2  POS      String
    Values#   colname  type
───────────────────
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [96]:
println(typeof(vcf_ndsparse_738["chr1", 58211]))

NDSparse{NamedTuple{(:ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{8,String}},Tuple{String},StructArrays.StructArray{NamedTuple{(:POS,),Tuple{String}},1,NamedTuple{(:POS,),Tuple{WeakRefStrings.StringArray{String,1}}},Int64},StructArrays.StructArray{NamedTuple{(:ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{8,String}},1,NamedTuple{(:ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM),NTuple{8,WeakRefStrings.StringArray{String,1}}},Int64}}


In [7]:
x = 0

for item in vcf_ndsparse_738
    
    println(item) 

    println(item[:REF])

    end
    
end

(ID = "ID", REF = "REF", ALT = "ALT", QUAL = "QUAL", FILTER = "FILTER", INFO = "INFO", FORMAT = "FORMAT", GERM = "Germ")
REF
(ID = ".", REF = "T", ALT = "C", QUAL = "136", FILTER = "PASS", INFO = "SNVHPOL=3;MQ=54", FORMAT = "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL", GERM = "0/1:169:17:36:0:22,14:12,11:10,3:-9.6:PASS:170,0,269")
T
(ID = ".", REF = "A", ALT = "G", QUAL = "696", FILTER = "PASS", INFO = "SNVHPOL=3;MQ=60", FORMAT = "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL", GERM = "1/1:144:30:49:1:0,49:0,19:0,30:-68.7:PASS:370,147,0")
A
(ID = ".", REF = "G", ALT = "T", QUAL = "455", FILTER = "PASS", INFO = "SNVHPOL=4;MQ=60", FORMAT = "GT:GQ:GQX:DP:DPF:AD:ADF:ADR:SB:FT:PL", GERM = "1/1:87:30:30:3:0,30:0,13:0,17:-49.8:PASS:370,90,0")
G
(ID = ".", REF = "ATTATTTATTTAT", ALT = "A", QUAL = "519", FILTER = "PASS", INFO = "CIGAR=1M12D;RU=TTAT;REFREP=13;IDREP=10;MQ=60", FORMAT = "GT:GQ:GQX:DPI:AD:ADF:ADR:FT:PL", GERM = "1/1:70:19:34:0,23:0,12:0,11:PASS:528,73,0")
ATTATTTATTTAT
(ID = ".", REF = "C", ALT =

LoadError: InterruptException:

In [5]:
vcf_ndsparse = loadndsparse(
    joinpath(input_dir, "738_variants.vcf"),    
    delim='\t',
    skiplines_begin=261,
    header_exists=false, 
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
    indexcols=[:CHROM, :POS]
)

LoadError: UndefVarError: input_dir not defined

In [ ]:
using Dates

start_time = now()

vcf_ndsparse["chr1", 28558]

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")